# Mosaic Data Augmentation on Image Dataset

This notebook demonstrates **Mosaic Data Augmentation**, a technique used to create new training images by combining four randomly selected images into one mosaic image. This augmentation strategy helps introduce varied backgrounds and complex object arrangements, enhancing the robustness of object detection models.

## Overview of the Process
1. **Select and Load Images**: Four images are randomly chosen from the input folder.
2. **Calculate Image Density**: The image with the highest density of annotated objects (relative to its area) is identified. This "densest" image will occupy the largest area in the final mosaic.
3. **Create Mosaic Image**:
    - An empty canvas of 3000x3000 pixels is initialized.
    - A random center point is selected to divide the canvas into four quadrants.
    - The densest image is placed in the largest quadrant, while the other three images fill the remaining quadrants.
4. **Save the Mosaic Image**: Each resulting mosaic image is saved to the specified output folder.

## Key Functions
- **calculate_density**: Computes the density of objects in each image based on annotations.
- **crop_center**: Crops the central region of an image for better placement within the mosaic.
- **mosaic_augmentation**: Combines the four images into a single mosaic, following the layout strategy described above.

This process provides a richer dataset by blending parts of different images, simulating more complex scenes for improved model training.

---

After this description, proceed to the code cells below to run the Mosaic Data Augmentation.


In [ ]:
import cv2
import numpy as np
import os
import random

def calculate_density(images, annotations_len):
    '''
    Calculate the density of an image
    input:
    images: list of images
    annotations_len: list of annotations lengths    
    output:     
    max_density_index: index of the image with the highest density
    '''
    densities = [annotations_len[i] / (images[i].shape[0] * images[i].shape[1]) for i in range(len(images))]
    max_density_index=np.argmax(densities)
    return max_density_index


def crop_center(image, target_height, target_width):
    '''
    crop the center of the image to the target size
    input:
    image: image
    target_height: target height 
    target_width: target width
    output: cropped image   

    '''
    h, w = image.shape[:2]
    start_x = (w - target_width) // 2
    start_y = (h - target_height) // 2
    return image[start_y:start_y + target_height, start_x:start_x + target_width]
def read_annotations(annotations_path):
    '''
    read annotations from the file
    input:
    annotations_path: path of the annotations file
    output: 
    len(lines): number of lines in the file(objects in the image)
    '''
    with open(annotations_path, 'r') as file:
        lines = file.readlines()[2:]
        # Skip the first two lines
    return len(lines)

def mosaic_augmentation(images, max_density_index,output_folder_annotation,counter):
    '''
    make mosaic augmentation
    input:
    images: list of images
    max_density_index: index of the image with the highest density
    output_folder_annotation: path of the output folder
    counter: counter
    output: 
    mosaic_img: mosaic image
    '''
    mosaic_img = np.zeros((3000, 3000, 3), dtype=np.uint8) #make empty mosaic image with size 3000x3000
    #claculate random point of image
    center_x = random.randint(300, 2700) #claculate x coordinates of random point

    center_y = random.randint(300, 2700) #claculate y coordinates of random point
    # calculate 4 quarters of the image
    quarters = [
        (center_x * center_y, (0, center_y, 0, center_x)),  # top-left
        ((3000 - center_x) * center_y, (0, center_y, center_x, 3000)),  # top-right
        (center_x * (3000 - center_y), (center_y, 3000, 0, center_x)),  # bottom-left
        ((3000 - center_x) * (3000 - center_y), (center_y, 3000, center_x, 3000))  # bottom-right
    ]
    
    larger_quarter = max(quarters, key=lambda q: q[0])[1]
    mosaic_img[larger_quarter[0]:larger_quarter[1], larger_quarter[2]:larger_quarter[3]] = cv2.resize(
        images[max_density_index],
        (larger_quarter[3] - larger_quarter[2], larger_quarter[1] - larger_quarter[0])
    )

   

    remaining_images = [images[i] for i in range(4) if i != max_density_index]

    # Calculate the positions of the other quarters if larger quarter is top-left
    if larger_quarter == quarters[0][1]:  # top-left
        positions = [
            (0, center_y, center_x, 3000),  # top-right
            (center_y, 3000, 0, center_x),   # bottom-left
            (center_y,3000,center_x,3000)
        ]
      

    # Calculate the positions of the other quarters if larger quarter is top-right

    elif larger_quarter == quarters[1][1]:  # top-right
        positions = [
            (0, center_y, 0, center_x),      # top-left
            (center_y, 3000, center_x, 3000),#bottom_right
            (center_y, 3000, 0, center_x)
              # bottom-right
        ]
   
    # Calculate the positions of the other quarters if larger quarter is bottom-left
    elif larger_quarter == quarters[2][1]:  # bottom-left
        positions = [
            (0, center_y, center_x, 3000),    # top-right
            (0, center_y, 0, center_x),#top_left
            (center_y,3000,center_x,3000)
                              
        ]
    # Calculate the positions of the other quarters if larger quarter is bottom-right
    else:  # bottom-right
        positions = [
            (0, center_y, 0, center_x),        # top-left
            (center_y, 3000, 0, center_x),      # bottom-left
            (0,center_y,center_x,3000)
        ]
   
# Crop the images and paste them into the mosaic image
    for img, pos in zip(remaining_images, positions):
        mosaic_img[pos[0]:pos[1], pos[2]:pos[3]] = crop_center(img, pos[1] - pos[0], pos[3] - pos[2])

    return mosaic_img


# files path
image_folder = 'path/input_image'
output_folder = 'path/output_image'
os.makedirs(output_folder, exist_ok=True)

counter = 0
while counter < 5:
    image_paths = random.sample([os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith('.jpg')], 4)
    images = [cv2.imread(img_path) for img_path in image_paths]
    annotations_len = [read_annotations(ann_path) for ann_path in annotation_paths]

# if all image load well
    if not all(isinstance(img, np.ndarray) for img in images):
        continue
    max_density = calculate_density(images,annotations_len)
    try:
        mosaic_image = mosaic_augmentation(images, max_density,output_folder_annotation,counter)
    except Exception as e:
        continue

    output_path = os.path.join(output_folder, f'mosaic_result_{counter}.jpg')
    cv2.imwrite(output_path, mosaic_image)
    counter += 1


# Cutout Data Augmentation on DOTA Dataset

This notebook demonstrates **Cutout Data Augmentation**, a technique for randomly obscuring parts of images by applying square cutouts. This process aids in making object detection models more robust by teaching them to recognize objects even when parts of the scene are hidden or missing.

## Overview of the Process
1. **Class Mapping Setup**: Defines the mapping between class names (e.g., "plane", "ship") and class IDs used in the annotations.
2. **Image and Label Loading**: The `load_image_and_labels` function loads an image and its corresponding labels (coordinates and class IDs) from text files.
3. **Applying Cutouts**:
    - The `cutout` function applies a random square cutout on the image. Any objects (annotations) that overlap with the cutout area are removed from the final labels.
    - The size and number of cutout regions can be specified; here, a single cutout region of 312x312 pixels is applied.
4. **Drawing Annotations**: The `draw_annotations` function overlays the bounding polygons and class labels onto the image for both the original and cutout-augmented images, enabling a visual comparison.
5. **Saving the Augmented Data**:
    - The modified image, updated labels, and annotated images are saved to specified folders for later review and training.

## Key Functions
- **load_image_and_labels**: Loads the image and associated annotations, mapping each class name to its class ID.
- **cutout**: Applies random cutouts to the image and removes annotations that overlap with the cutout region.
- **draw_annotations**: Draws bounding polygons and class labels on the image.

This approach is beneficial for datasets with varied object types and placements, as it introduces challenges that strengthen model performance in real-world scenarios.

---

You can now proceed to run the code cells to see the cutout augmentation in action on the DOTA dataset.


In [ ]:
import os
import random
import cv2
import numpy as np

# Mapping for class labels
class_mapping = {
    'plane': 0,
    'ship': 1,
    'storage-tank': 2,
    'baseball-diamond': 3,
    'tennis-court': 4,
    'basketball-court': 5,
    'ground-track-field': 6,
    'harbor': 7,
    'bridge': 8,
    'large-vehicle': 9,
    'small-vehicle': 10,
    'helicopter': 11,
    'roundabout': 12,
    'soccer-ball-field': 13,
    'swimming-pool': 14,
    'container-crane': 15
}

# Invert class mapping for easy access
class_names = {v: k for k, v in class_mapping.items()}

def load_image_and_labels(image_path, label_path):
    '''
    Load image and labels from a text file
    input: image_path, label_path
    output: image, labels

    '''
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Failed to load image at {image_path}") # Raise an exception if the image could not be loaded
    
    labels = []
    with open(label_path, 'r') as file:
        content = file.readlines()
    
    for line in content[1:]:  # Skip the first and second lines
        if line.startswith("imagesource") or line.startswith("gsd"):
            continue
        
        parts = line.strip().split()
        coords = list(map(float, parts[:-2]))
        class_name = parts[-2]
        class_id = class_mapping.get(class_name)
        
        if class_id is None:
            raise ValueError(f"Class '{class_name}' not found in the class mapping.")
        
        labels.append(coords + [class_id])

    return image, np.array(labels)

def cutout(image, labels, n_holes=1, length=512):
    '''
    make a cutout on the image
    input: image, labels, n_holes, length
    output: image, labels, deleted_annotations
    
    '''

    h, w, _ = image.shape
    new_labels = []
    deleted_annotations = []  # List to keep track of deleted annotations

    for _ in range(n_holes):
        # Randomly select the position to cut out
        y = np.random.randint(0, h)
        x = np.random.randint(0, w)

        # Calculate the square coordinates
        y1 = np.clip(y - length // 2, 0, h)
        y2 = np.clip(y + length // 2, 0, h)
        x1 = np.clip(x - length // 2, 0, w)
        x2 = np.clip(x + length // 2, 0, w)

        # Apply cutout
        image[y1:y2, x1:x2] = 0
        
        # Check for overlaps with annotations and remove them
        for label in labels:
            coords = label[:-1]  # Get coordinates of the annotation
            if (
                (coords[0] < x2 and coords[2] > x1 and coords[1] < y2 and coords[5] > y1)
            ):  # Check if there's an overlap
                deleted_annotations.append(label)  # Store deleted annotation
                continue  # Skip this annotation if it overlaps
            new_labels.append(label)  # Keep the annotation if it doesn't overlap
    
    return image, np.array(new_labels), deleted_annotations


def draw_annotations(image, labels, color=(0, 255, 0), thickness=2):
    '''
    draw annotations on the image
    input: image, labels, color, thickness
    output: image
    '''
    for label in labels:
        coords = np.array(label[:-1], dtype=np.int32).reshape((-1, 2))
        class_id = int(label[-1])
        class_name = class_names[class_id]
        cv2.polylines(image, [coords], isClosed=True, color=color, thickness=thickness)
        cv2.putText(image, class_name, (coords[0][0], coords[0][1] - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

# Dataset paths
database_folder = 'path/train/image'
label_folder = 'path/train/annotation'



# Get list of image files
image_files = [f for f in os.listdir(database_folder) if f.endswith('.png')]

# Number of images to generate
num_images_to_generate = 100

for i in range(num_images_to_generate):
    random_file = random.choice(image_files)

    # Load image and labels
    image, labels = load_image_and_labels(
        os.path.join(database_folder, random_file), 
        os.path.join(label_folder, random_file.replace('.png', '.txt'))
    )

    # Draw annotations on the original image for comparison
    original_annotated_image = image.copy()
    draw_annotations(original_annotated_image, labels)

    # Apply cutout
    cutout_image, new_labels, deleted_annotations = cutout(image.copy(), labels, n_holes=1, length=312)

    # Draw annotations on cutout image
    annotated_image = cutout_image.copy()
    draw_annotations(annotated_image, new_labels)

    # Save the cutout image and labels
    output_image_path = f'path/save/image/cutoutpic-3{i + 1}.jpg'
    output_labels_path = f'path/save/label/cutoutpic-3{i + 1}.txt'
    output_annotated_image_path = f'path/save/image_annotate/cutoutpic-3{i + 1}.jpg'
    
 

    # Save cutout image
    cv2.imwrite(output_image_path, cutout_image)

    # Save new annotations
    np.savetxt(output_labels_path, new_labels, fmt='%.2f', delimiter=' ', header='x1 y1 x2 y2 x3 y3 x4 y4 class', comments='')

    # Save annotated cutout image
    cv2.imwrite(output_annotated_image_path, annotated_image)

    # # Save original annotated image
    # cv2.imwrite(original_annotated_image_path, original_annotated_image)


    print(f"Cutout image saved at: {output_image_path}")
    print(f"Cutout labels saved at: {output_labels_path}")
    print(f"Annotated cutout image saved at: {output_annotated_image_path}")
